# 네이버 영화 기사 수 크롤링

In [1]:
import pandas as pd
import json
import os
import re
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import Series, DataFrame
import datetime
import time

In [2]:
MovieData=pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\movie_final_data1.csv', encoding='CP949')
news_date=pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\news_date.csv', encoding='CP949')
special_score=pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\special_score.csv', encoding='CP949')
red_date = pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\red_date.csv', encoding='CP949')

red_date.columns=['startRd','endRd']
red_date

,startRd,endRd
0,2013-12-18,2014-01-01
1,2014-01-16,2014-01-30
2,2014-01-17,2014-01-31
3,2014-01-18,2014-02-01
4,2014-02-15,2014-03-01
...,...,...
83,2018-09-11,2018-09-25
84,2018-09-12,2018-09-26
85,2018-09-19,2018-10-03
86,2018-09-25,2018-10-09


In [138]:
for i in range(0,88):
    red_date['startRd'][i] = datetime.datetime.strptime(red_date['startRd'][i],"%Y-%m-%d").date()
    red_date['endRd'][i] = datetime.datetime.strptime(red_date['endRd'][i],"%Y-%m-%d").date()
red_date

,startRd,endRd
0,2013-12-18,2014-01-01
1,2014-01-16,2014-01-30
2,2014-01-17,2014-01-31
3,2014-01-18,2014-02-01
4,2014-02-15,2014-03-01
...,...,...
83,2018-09-11,2018-09-25
84,2018-09-12,2018-09-26
85,2018-09-19,2018-10-03
86,2018-09-25,2018-10-09


In [187]:
for i in range(0,817):
    MovieData['openDt'][i] = datetime.datetime.strptime(MovieData['openDt'][i],"%Y-%m-%d").date()

TypeError: strptime() argument 1 must be str, not datetime.date

In [188]:
specialscore=special_score['moviespecialScore']
specialscore

0      5.69
1      5.00
2      5.50
3      8.08
4      5.00
       ... 
811    4.57
812    5.17
813    6.75
814    5.63
815    5.50
Name: moviespecialScore, Length: 816, dtype: float64

In [140]:
Movies=[i for i in MovieData['movieNm']]
Search=zip(Movies)

In [141]:
Plus=[]
for i in Search:
    a=list(i)
    Plus.append(a)
    

MovieDirector=[]
for i in Plus:
    b=' '.join(i)
    MovieDirector.append(b)

MovieDirector = DataFrame(MovieDirector)

In [142]:
#검색리스트와 기간을 url에 넣을 수 있는 함수
def URLmaker(query,sdate,edate):
    try:
        base_url = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}'
        encoded_query=requests.utils.quote(query, encoding='MS949')     
        base_url = base_url.format(encoded_query, sdate, edate)
    except:
        print(i)
        pass
    return base_url

In [143]:
#앞에 만든 함수에 검색과 기간을 넣어서 최종 url 생성
FinalUrl=[]
for i in range(len(MovieDirector)):
    url=URLmaker(MovieDirector[0][i], news_date['startDt'][i], news_date['openDt'][i])
    FinalUrl.append(url)

In [72]:
FinalUrl[3]

'https://search.naver.com/search.naver?where=news&query=1987&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2017.12.13&de=2017.12.27'

In [86]:
news_count=[]
num=0

def get_tag(i):
        rep = requests.get(i)
        source = rep.text
        soup = BeautifulSoup(source, 'html.parser')
        #print(soup.select("#main_pack > div > div.section_head > div.title_desc.all_my > span"))
        tag = str(soup.select("#main_pack > div > div.section_head > div.title_desc.all_my > span"))
        tag_split = tag[14:]
        tag_split = tag_split[0:-9]
        print(tag_split)
        return tag_split
    
for i in FinalUrl:
    news_count.append(get_tag(i))
    
news_count=pd.DataFrame(news_count)
news_count

629
54
172
3,846
41
1,096
136
218
45
50
34
2,437
41
1,614
180
306
1,726
4,914
2,606
11
44
4,419
2,741
2,166
14
33
67
1,256
3,372
2,979
3,321
80
462
1,881
1,624
98
2,420
96
100
3,620
889
1,626
164
2,334
820
4,430
6,131
1,960
222
43
6,149
1,620
2,874
4,057
5,511
458
695
1,567
769
10,806
3,945
451
3,356
6,974
2,239
68
49
394
91
64
1,073
3,043
17
18
22
14
15
12
21

56
10
16
10
69
16
23
11
10


10
32
10
10
86
1,786
2,570
26
21
3,301
2,728
559
48
3,517
2,628
53
30
4,912
2,863
48
204
138
312
46
1,252
47
3,081
36
293
2,488
1,866
2,274
2,107
2,933
853
2,580
55
2,036
1,387
26
1,166
1,585
347
1,410
94
425

10,604
39
426
43
384
14
195
22,408
272
402
27
2,722
119
200
43
45
28
90
35
217
256
52
572
812
894
168
2,499
29
69
2,824
1,378
1,288
2,739
65
44
563
96
38
802
45
124
764
108
93
82
152
3,530
23
2,207
2,135
74
711
2,253
3,007
75
549
355
1,822
44
39
34
1,133
139
519
3,786
978
3,585
110
2,117
22
2,531
2,001
1,901
2,194
2,815
47
208
102
53
1,019
59
55
1,251
204
36
42
1,058
23
6,542
42
208
48
11
31
45

,0
0,629
1,54
2,172
3,"3,846"
4,41
...,...
811,243
812,"1,030"
813,148
814,"3,934"


In [144]:
news_count.columns=['news']
news_count

,news
0,629
1,54
2,172
3,3846
4,41
...,...
811,243
812,1030
813,148
814,3934


In [145]:
type(news_count['news'][0])

str

In [146]:
for i in range(0,816):
    news_count['news'][i] = news_count['news'][i].replace(',', '')
news_count

,news
0,629
1,54
2,172
3,3846
4,41
...,...
811,243
812,1030
813,148
814,3934


In [181]:
redday = pd.DataFrame(np.zeros((816,1)))
redday.columns=['redday']
redday

,redday
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
811,0.0
812,0.0
813,0.0
814,0.0


In [182]:
for i in range(0,816):
    for j in range(0,88):
        if(abs(MovieData['openDt'][i]-red_date['startRd'][j]).days <=14 and abs(MovieData['openDt'][i]-red_date['endRd'][j]).days <=14):
            redday['redday'][j]='Y'
redday

,redday
0,0
1,Y
2,Y
3,Y
4,Y
...,...
811,0
812,0
813,0
814,0


In [183]:
for i in range(0,816):  
    redday['redday'][i] = str(redday['redday'][i])
    redday['redday'][i] = redday['redday'][i].replace('0.0','N')
redday

,redday
0,N
1,Y
2,Y
3,Y
4,Y
...,...
811,N
812,N
813,N
814,N


In [184]:
MovieData=pd.concat([MovieData,news_count,specialscore,redday],axis=1)
MovieData

,movieNm,movieCd,openDt,audiCnt,audiAcc,scrnCnt,showCnt,date,salesAmt,salesShare,...,actorscore,directorscore,distributorscore,codes,like,days,weeks,news,moviespecialScore,redday
0,007 스펙터,20157432,2015-11-11,79058,1344283,1039,4509,2015-11-17,586160800,30.6,...,85091672,39543680,21346321,99701,6919,7,1,629,5.69,N
1,12 솔져스,20178395,2018-01-31,11378,154107,211,762,2018-02-06,91873900,5.6,...,716835469,3017882,21346321,168037,937,7,1,54,5.00,Y
2,13시간,20161630,2016-03-03,8741,124249,359,970,2016-03-09,66523800,4.5,...,28796897,253981901,21346321,140265,770,7,1,172,5.50,Y
3,1987,20170590,2017-12-27,217247,2690793,1063,5044,2018-01-02,1643436600,33.1,...,3378295278,338451089,21346321,158191,13877,7,1,3846,8.08,Y
4,19곰 테드 2,20156749,2015-06-25,13680,133198,280,800,2015-07-01,105966600,3.7,...,262591058,2344077,21346321,104426,2859,7,1,41,5.00,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,흥부: 글로 세상을 바꾼 자,20176121,2018-02-14,16983,359353,540,1618,2018-02-20,126100600,4.9,...,753458618,5145958,21346321,159805,4290,7,1,243,4.57,N
812,희생부활자,20154907,2017-10-12,18054,300737,606,2352,2017-10-18,129406600,6.8,...,184082276,90514065,21346321,140696,5472,7,1,1030,5.17,N
813,히든 피겨스,20167363,2017-03-23,20436,196427,356,713,2017-03-29,121064400,4.1,...,29940608,15315806,370035687,147092,1921,7,1,148,6.75,N
814,히말라야,20136068,2015-12-16,180295,1896771,1007,4870,2015-12-22,1324284300,42.5,...,4315790217,1011506559,21346321,100647,13693,7,1,3934,5.63,N


In [190]:
MovieData['movieNm'][637]

'조선명탐정 : 사라진 놉의 딸'

In [186]:
MovieData.to_csv('C:\\Users\\yoonj\Desktop\\regression\\data\\MovieData.csv',index=False,encoding='utf-8-sig')

-------------------------------

In [259]:
MovieData = pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\MovieData.csv', encoding='utf-8-sig')
sataudi = pd.read_csv(r'C:\\Users\\yoonj\Desktop\\regression\\data\\Sat_audicnt.csv', encoding='CP949')
sataudi

,movieNm,audiSat
0,007 스펙터,356331
1,12 솔져스,35409
2,13시간,35658
3,1987,487098
4,19곰 테드 2,26466
...,...,...
811,흥부: 글로 세상을 바꾼 자,79863
812,희생부활자,74141
813,히든 피겨스,44551
814,히말라야,422443


In [258]:
#정글북 -> 엑셀로 삭제
#sataudi=sataudi.drop(sataudi.index[636])
#sataudi.reset_index()

KeyError: 637

In [260]:
audiSat=sataudi['audiSat']
audiSat

0      356331
1       35409
2       35658
3      487098
4       26466
        ...  
811     79863
812     74141
813     44551
814    422443
815     13277
Name: audiSat, Length: 816, dtype: int64

In [261]:
MovieData=pd.concat([MovieData,audiSat],axis=1)
MovieData

,movieNm,movieCd,openDt,audiCnt,audiAcc,scrnCnt,showCnt,date,salesAmt,salesShare,...,directorscore,distributorscore,codes,like,days,weeks,news,moviespecialScore,redday,audiSat
0,007 스펙터,20157432,2015-11-11,79058,1344283,1039,4509,2015-11-17,586160800,30.6,...,39543680,21346321,99701,6919,7,1,629.0,5.69,N,356331
1,12 솔져스,20178395,2018-01-31,11378,154107,211,762,2018-02-06,91873900,5.6,...,3017882,21346321,168037,937,7,1,54.0,5.00,Y,35409
2,13시간,20161630,2016-03-03,8741,124249,359,970,2016-03-09,66523800,4.5,...,253981901,21346321,140265,770,7,1,172.0,5.50,Y,35658
3,1987,20170590,2017-12-27,217247,2690793,1063,5044,2018-01-02,1643436600,33.1,...,338451089,21346321,158191,13877,7,1,3846.0,8.08,Y,487098
4,19곰 테드 2,20156749,2015-06-25,13680,133198,280,800,2015-07-01,105966600,3.7,...,2344077,21346321,104426,2859,7,1,41.0,5.00,Y,26466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,흥부: 글로 세상을 바꾼 자,20176121,2018-02-14,16983,359353,540,1618,2018-02-20,126100600,4.9,...,5145958,21346321,159805,4290,7,1,243.0,4.57,N,79863
812,희생부활자,20154907,2017-10-12,18054,300737,606,2352,2017-10-18,129406600,6.8,...,90514065,21346321,140696,5472,7,1,1030.0,5.17,N,74141
813,히든 피겨스,20167363,2017-03-23,20436,196427,356,713,2017-03-29,121064400,4.1,...,15315806,370035687,147092,1921,7,1,148.0,6.75,N,44551
814,히말라야,20136068,2015-12-16,180295,1896771,1007,4870,2015-12-22,1324284300,42.5,...,1011506559,21346321,100647,13693,7,1,3934.0,5.63,N,422443


In [262]:
MovieData.to_csv('C:\\Users\\yoonj\Desktop\\regression\\data\\final_data.csv',index=False,encoding='CP949')